# VacationPy
----


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint
# Import API key
from config import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv('output_data/cities.csv')
cities_df.head()


,city,lat,lng,max temperature,humidity,cloudiness,wind speed,country,date
0,new norfolk,-42.780200,147.061533,60.53,51.0,3.0,9.22,AU,1.659577e+09
1,hobart,-42.882509,147.328123,60.60,62.0,20.0,8.05,AU,1.659577e+09
2,ca mau,9.179221,105.145791,83.16,77.0,100.0,7.07,VN,1.659577e+09
3,sayat,45.827315,3.051024,73.42,46.0,0.0,1.14,FR,1.659577e+09
4,rikitea,-23.122954,-134.968014,71.80,72.0,93.0,18.68,PF,1.659577e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = cities_df[['lat', 'lng']]
humidity = cities_df['humidity'].astype(float)

In [4]:
# create gmaps figure
fig = gmaps.figure()

# creating heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Max Temp lower than 80 but higher than 70
# Wind speed less than 10 mph
# Cloudiness less than or equal to 50
# using .loc
ideal_weather_df = cities_df.loc[(cities_df['max temperature'] < 80) & (cities_df['max temperature'] >= 70) & (cities_df['wind speed'] < 10) & (cities_df['cloudiness'] <= 50)]
ideal_weather_df

,city,lat,lng,max temperature,humidity,cloudiness,wind speed,country,date
3,sayat,45.827315,3.051024,73.42,46.0,0.0,1.14,FR,1.659577e+09
17,castro,45.802987,10.067060,77.97,72.0,9.0,5.99,IT,1.659577e+09
48,lagoa,37.132581,-8.455051,72.88,78.0,0.0,5.61,PT,1.659577e+09
52,praia da vitoria,38.739238,-27.156264,72.63,88.0,20.0,1.81,PT,1.659577e+09
53,vila franca do campo,37.717162,-25.433535,73.81,93.0,38.0,5.12,PT,1.659577e+09
66,guiratinga,-16.345742,-53.759510,72.00,36.0,4.0,4.27,BR,1.659577e+09
70,albany,42.651167,-73.754968,78.15,69.0,0.0,1.28,US,1.659577e+09
84,alta floresta,-9.871502,-56.083988,74.93,42.0,22.0,2.30,BR,1.659577e+09
110,springfield,39.799017,-89.643957,75.16,89.0,0.0,5.75,US,1.659577e+09
160,takoradi,4.887401,-1.751932,72.28,81.0,50.0,9.26,GH,1.659577e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_weather_df.copy()
hotel_df['Hotel Name'] = ""
hotel_df = hotel_df.drop(columns=['date'])
hotel_df

,city,lat,lng,max temperature,humidity,cloudiness,wind speed,country,Hotel Name
3,sayat,45.827315,3.051024,73.42,46.0,0.0,1.14,FR,
17,castro,45.802987,10.067060,77.97,72.0,9.0,5.99,IT,
48,lagoa,37.132581,-8.455051,72.88,78.0,0.0,5.61,PT,
52,praia da vitoria,38.739238,-27.156264,72.63,88.0,20.0,1.81,PT,
53,vila franca do campo,37.717162,-25.433535,73.81,93.0,38.0,5.12,PT,
66,guiratinga,-16.345742,-53.759510,72.00,36.0,4.0,4.27,BR,
70,albany,42.651167,-73.754968,78.15,69.0,0.0,1.28,US,
84,alta floresta,-9.871502,-56.083988,74.93,42.0,22.0,2.30,BR,
110,springfield,39.799017,-89.643957,75.16,89.0,0.0,5.75,US,
160,takoradi,4.887401,-1.751932,72.28,81.0,50.0,9.26,GH,


In [7]:
# Loop through Hotel Dataframe
# Using google places API nearbysearch
# Setting up base url and parameters
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
parameters = {
    'location': "",
    'radius': 5000,
    'type': 'lodging',
    'key': g_key,
}

for index, row in hotel_df.iterrows():
    # grab location
    lat = row['lat']
    lng = row['lng']
    
    # put location into parameters
    parameters['location'] = f'{lat},{lng}'
    
    # get json response for each location
    print(f'retrieving results for {index}: {lat},{lng}')
    response = requests.get(base_url, params=parameters).json()
    result = response['results']
#     pprint(result)
    try:
        print('hotel found')
        name = result[0]['name']
        print(f'Nearest Hotel: {name}')
        
        hotel_df.loc[index, 'Hotel Name'] = result[0]['name']
        
    except (KeyError, IndexError):
        print('Hotels not found')


retrieving results for 3: 45.8273147,3.0510236
hotel found
Nearest Hotel: Chateau de Féligonde
retrieving results for 17: 45.802987,10.067060000000001
hotel found
Nearest Hotel: Hotel Lovere Resort & Spa
retrieving results for 48: 37.13258125,-8.455050948117309
hotel found
Nearest Hotel: DETAILS Vale da Lapa Village Resort
retrieving results for 52: 38.7392376,-27.156264396780454
hotel found
Nearest Hotel: Sawmill Azores
retrieving results for 53: 37.7171624,-25.433535199999998
hotel found
Nearest Hotel: Pestana Bahia Praia
retrieving results for 66: -16.3457422,-53.75950970000001
hotel found
Nearest Hotel: Hotel Arcoverde
retrieving results for 70: 42.6511674,-73.754968
hotel found
Nearest Hotel: Hilton Albany
retrieving results for 84: -9.8715018,-56.083988100000006
hotel found
Nearest Hotel: Floresta Amazonica Hotel
retrieving results for 110: 39.7990175,-89.6439575
hotel found
Nearest Hotel: President Abraham Lincoln Springfield - a DoubleTree by Hilton Hotel
retrieving results for

In [8]:
# Droping Rows with no Hotels 
# Replacing blank('') with np.nan
# Drop na
hotel_df
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df

,city,lat,lng,max temperature,humidity,cloudiness,wind speed,country,Hotel Name
3,sayat,45.827315,3.051024,73.42,46.0,0.0,1.14,FR,Chateau de Féligonde
17,castro,45.802987,10.067060,77.97,72.0,9.0,5.99,IT,Hotel Lovere Resort & Spa
48,lagoa,37.132581,-8.455051,72.88,78.0,0.0,5.61,PT,DETAILS Vale da Lapa Village Resort
52,praia da vitoria,38.739238,-27.156264,72.63,88.0,20.0,1.81,PT,Sawmill Azores
53,vila franca do campo,37.717162,-25.433535,73.81,93.0,38.0,5.12,PT,Pestana Bahia Praia
66,guiratinga,-16.345742,-53.759510,72.00,36.0,4.0,4.27,BR,Hotel Arcoverde
70,albany,42.651167,-73.754968,78.15,69.0,0.0,1.28,US,Hilton Albany
84,alta floresta,-9.871502,-56.083988,74.93,42.0,22.0,2.30,BR,Floresta Amazonica Hotel
110,springfield,39.799017,-89.643957,75.16,89.0,0.0,5.75,US,President Abraham Lincoln Springfield - a Doub...
160,takoradi,4.887401,-1.751932,72.28,81.0,50.0,9.26,GH,Raybow International Hotel


In [9]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))